<a href="https://colab.research.google.com/github/sasankgamini/GDSCProject/blob/main/nightmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api --upgrade
!pip install folium --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.3 MB/s 
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1


In [ ]:
import ee
import folium



In [ ]:
ee.Authenticate()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=A83pkUhHr-jBKI659MGf32d0vD6Ep7aMaPnMErSgXHU&tc=29u-AnfUbahbD4utovze0IrUz1m7aFOo4MyIDTkCNig&cc=KiSWm8FtGXDjn9Ju_J5N4VRNjwGBKMnUAXdbHE_5Ukc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdcajWk_cXWuoqFKMFuVYY0UPAcUuvw_Ef3rOlW8rMBCwgoy-vEFXZg

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
#Loading the Night time Collection
collection = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')
# Extract the first image
first_image = collection.first()

# Display band information about the image
first_image.bandNames().getInfo()

collection = collection.select('stable_lights')


first_image = collection.first()
 
# Get number of years after 1991 from the first image
year = ee.Date(first_image.get('system:time_start')).get('year').subtract(1991)
 
# Turn it into a band
img1 = ee.Image(year).byte().addBands(first_image)
 
# A new band appears in the new image
img1.bandNames().getInfo()

def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)
 
# Map it to the collection to create a new collection
collection = collection.map(createTimeBand)


In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
 
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create the night map
night_map = folium.Map(location=[37.5010, -102.1899], zoom_start=4.5)

#  visualize the y-intercept in green, and positive/negative slopes as red/blue.
params = {
    'bands' :  ['scale', 'offset', 'scale'],
    'min' : 0,
    'max' : [0.18, 20, -0.18],
}

night_light = collection.reduce(ee.Reducer.linearFit())

# Add the image layer to the night_map
night_map.add_ee_layer(night_light, params, 'test')
 
# Display the map
display(night_map)

